In [101]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
import pandas as pd
from src.data_processing.processors.guppy_processors.DataLoader import DataLoader
from src.data_processing.processors.guppy_processors.PavProcessor import PavProcessor
from src.data_processing.processors.guppy_processors.PavVisualizer import PavVisualizer
from src.data_processing.processors.guppy_processors.ExperimentMetaData import ExperimentMetaData

# def run_analysis():
# CONFIG_PATH = '/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/conf/config.yaml'
# data_loader = DataLoader(CONFIG_PATH)
# pav_processor = PavProcessor(meta_data)
data_loader

In [115]:
experiment_path = '/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1/4756-231201-122549'
gup = ExperimentMetaData(experiment_path)

In [125]:
gup.stores_lists_as_frame['output_1']

,405A,405C,465A,465C,Cuet,Lick,Rwrp,endr
0,control_NAC,control_LHA,signal_NAC,signal_LHA,cue,lick,reward,encoder


In [130]:
hdf5_files = list(f for f in gup.guppy_paths['output_paths'][0].iterdir() if 'hdf5' in f.name)


[PosixPath('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1/4756-231201-122549/4756-231201-122549_output_1/cue_NAC.hdf5'),
 PosixPath('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1/4756-231201-122549/4756-231201-122549_output_1/465C.hdf5'),
 PosixPath('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1/4756-231201-122549/4756-231201-122549_output_1/lick_LHA.hdf5'),
 PosixPath('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1/4756-231201-122549/4756-231201-122549_output_1/reward_NAC.hdf5'),
 PosixPath('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1/4756-231201-122549/4756-231201-122549_output_1/405A.hdf5'),
 PosixPath('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopam

In [151]:
encoder_path = Path('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1/4755-231031-125328/4755-231031-125328_output_1/encoder_NAC.hdf5')

In [155]:
import h5py
with h5py.File(encoder_path, 'r') as f:
    print(list(f.keys()))

['ts']


In [156]:
df = pd.read_hdf(encoder_path, key = 'ts')
df

TypeError: cannot create a storer if the object is not existing nor a value are passed